In [15]:
import os
import shutil
import pathlib
import sys
ROOT = pathlib.Path('/notebooks/code/volumes/checkpoints/weights')

In [13]:
!git clone https://github.com/thekevinscott/image-super-resolution.git
!mv image-super-resolution/ISR image-super-resolution/IISR
sys.path.insert(0,"/notebooks/code/notebooks/image-super-resolution")

Cloning into 'image-super-resolution'...
remote: Enumerating objects: 1400, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 1400 (delta 48), reused 41 (delta 24), pack-reused 1335
Receiving objects: 100% (1400/1400), 14.98 MiB | 10.75 MiB/s, done.
Resolving deltas: 100% (752/752), done.


In [44]:
!rm -rf ./weights2
!rm -rf ./weights
!rm weights2.zip
!rm weights.zip

from IISR.models import RDN, RRDN
from tqdm import tqdm
import tensorflow as tf

def get_model(model, arch_params):
    if model == 'rdn':
        return RDN(arch_params=arch_params)
    elif model == 'rrdn':
        return RRDN(arch_params=arch_params)
    raise Exception('No valid model found for ' + model)
    
def get_params(folder):
    arch, C, D, G, G0, T, x, _, _2, _3, _4, _5 = folder.split('-')

    arch_params = {
        'C': int(C[1:]),
        'D': int(D[1:]),
        'G': int(G[1:]),
        'G0':int(G0[2:]),
        'x':int(x[1:])
    }
    if arch == 'rrdn':
        arch_params['T'] = int(T[1:])
    return arch, arch_params

# def save_model(weights, output, arch, x, C, D, G, G0, T):
#     model.model.load_weights('/code/weights/' + weights)
#     model.model.save('/code/weights/' + output)

def get_weights(folder):
    weights = []
    for date_folder in os.listdir(folder):
        date_folder = folder / date_folder
        weights += [str(date_folder / f) for f in os.listdir(date_folder) if 'srgan' not in f and f.endswith('hdf5')]
    return weights

def make_weights_zip(root, target):
    print(f'make weights.zip file for folder {root}')
    weights = []
    errs = []

    for folder in os.listdir(root):
        arch, arch_params = get_params(folder)
        weights += [(w, arch, arch_params) for w in get_weights(root / folder)]
        
    # weights = [w for w in weights if 'rrdn-C4-D3-G32-G064-T10-x4-patchsize128-compress100-sharpen0-datadiv2k-vary_cFalse_epoch491.hdf5' in w[0]]
    weights = weights[0:]
        
    i = 0
    for weight, arch, arch_params in tqdm(weights):
        try:
            tf.keras.backend.clear_session() # needed for https://github.com/tensorflow/tfjs/issues/755#issuecomment-489665951
            model = get_model(arch, arch_params)
            model.model.load_weights(weight)
            weight_name = weight.split('/')[-3:]
            weight_name = '/'.join(weight_name).split('.')[0] + '.h5'
            target_path = target / weight_name
            os.makedirs('/'.join(str(target_path).split('/')[0:-1]), exist_ok=True)
            model.model.save(target_path)       
            i += 1
        except Exception as e:
            errs += [(weight, e)]
            
    print(f'Successfully processed {i} files')
    if len(errs) > 0:
        print(f'The following {len(errs)} weights could not be processed\n-------------------------------')
        for err, e in errs:
            print(err, e)

    print('Start making zip file')
    shutil.make_archive(root, 'zip', './')
    print('Ready for download')
    
folder_name = 'weights'
try:
    os.remove(f'./{folder_name}.zip')
except:
    pass
try:
    shutil.rmtree(f'./{folder_name}')
except:
    pass
make_weights_zip(ROOT, pathlib.Path(f'./{folder_name}'))

rm: cannot remove 'weights.zip': No such file or directory
make weights.zip file for folder /notebooks/code/volumes/checkpoints/weights


100%|██████████| 225/225 [17:40<00:00,  4.71s/it]


Successfully processed 224 files
The following 1 weights could not be processed
-------------------------------
/notebooks/code/volumes/checkpoints/weights/rrdn-C4-D3-G32-G064-T10-x3-patchsize129-compress50-sharpen0-datadiv2k-vary_cTrue/2022-09-29_2348/rrdn-C4-D3-G32-G064-T10-x3-patchsize129-compress50-sharpen0-datadiv2k-vary_cTrue_best-val_generator_PSNR_Y_epoch191.hdf5 Unable to open file (unable to open file: name = '/notebooks/code/volumes/checkpoints/weights/rrdn-C4-D3-G32-G064-T10-x3-patchsize129-compress50-sharpen0-datadiv2k-vary_cTrue/2022-09-29_2348/rrdn-C4-D3-G32-G064-T10-x3-patchsize129-compress50-sharpen0-datadiv2k-vary_cTrue_best-val_generator_PSNR_Y_epoch191.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Start making zip file
Ready for download
